In [ ]:
# Define the example data
week = 46
month = 11
monat = 'November'
HotelName = 'Aparthotel Adagio Frankfurt City Messe'
City = 'Frankfurt'
Street = 'Hamburger Allee 4 '
id = 1758394 # this is hotel_id

In [ ]:
# Define some Methods for filtering the JSON Data
# JSON Data will contain customer data
# The JSON Data will be loaded via API below
def scoring(data):
    score=data["average_score"]
    d_score = round((score/4)*10)
    return d_score #score
def dcheckin(data):
    checkin = data["checkin"]
    d_checkin = datetime.strptime(checkin, "%Y-%m-%d").date()
    return d_checkin #check-in date
def dcheckout(data):
    checkout = data["checkout"]
    d_checkout = datetime.strptime(checkout, "%Y-%m-%d").date()
    return d_checkout
def dnights(data):
    d_nights = data["num_nights"]
    return d_nights #number of nights stayed
def ddate(data):
    date = data.split()[0]
    d_date = datetime.strptime(date, "%Y-%m-%d").date()
    return d_date #date when the review was written
def dcount(data):
    d_count_date=ddate(data["date"])-dcheckout(data["stayed_room_info"])
    return d_count_date.days #days from checkout to reivew
def custom_type(data):
    d_customer_type=data
    return d_customer_type #customer type
def country(data):
    d_country=data["countrycode"]
    return d_country # country
def travel_purpose(data):
    d_travel_purpose=data["travel_purpose"]
    return d_travel_purpose #travel purpose
def room_name(data):
    d_room_name = data["room_name"]
    return d_room_name #room name/category
def positiv(data):
    d_pro=data
    return d_pro
def negativ(data):
    d_neg=data
    return d_neg

In [ ]:
# Al important Libraries and HTTP Header
import http.client
import json
from datetime import datetime
import pandas as pd 
import matplotlib.pyplot as plt 

conn = http.client.HTTPSConnection("booking-com15.p.rapidapi.com")

key = "PUT HERE YOUR CODE"

headers = {
    'x-rapidapi-key': key,
    'x-rapidapi-host': "booking-com15.p.rapidapi.com"
}

In [ ]:
# Fetch all customer data for november.2024 and october.2024
# here for example we wanted only data from November and October
# this was written in november.2024
# thats why the condition is month-2
# but the data can also contain older values,
# since one batch of request comes with 25 customer reviews.
response=[]
i=1
while True:
    conn.request("GET", "/api/v1/hotels/getHotelReviews?hotel_id="+str(id)+"&sort_option_id=sort_recent_desc&page_number="+str(i), headers=headers)
    res = conn.getresponse()
    data = res.read()
    response.append(json.loads(data.decode("utf-8")))
    date = ddate(response[i-1]["data"]["result"][-1]["date"])
    if date.month <= month-2: # this only interested in values oct and nov
        break
    else:
        i += 1

In [ ]:
# We will start constructing a dataframe
# for converting the JSON
scores=[]
checkin=[]
checkout=[]
nights=[]
com_date=[]
count_date=[]
customer_type=[]
d_country=[]
trav_purpose=[]
room_names=[]
pos=[]
neg=[]
for i in range(len(response)):
    for j in range(len(response[i]["data"]["result"])):
        scores.append(scoring(response[i]["data"]["result"][j]))
        checkin.append(dcheckin(response[i]["data"]["result"][j]["stayed_room_info"]))
        checkout.append(dcheckout(response[i]["data"]["result"][j]["stayed_room_info"]))
        nights.append(dnights(response[i]["data"]["result"][j]["stayed_room_info"]))
        com_date.append(ddate(response[i]["data"]["result"][j]["date"]))
        count_date.append(dcount(response[i]["data"]["result"][j]))
        customer_type.append(custom_type(response[i]["data"]["result"][j]["author"]["type_string"]))
        d_country.append(country(response[i]["data"]["result"][j]["author"]))
        trav_purpose.append(travel_purpose(response[i]["data"]["result"][j]))
        room_names.append(room_name(response[i]["data"]["result"][j]["stayed_room_info"]))
        pos.append(positiv(response[i]["data"]["result"][j]["pros"]))
        neg.append(positiv(response[i]["data"]["result"][j]["cons"]))

In [ ]:
# create the dataframe
df_1_base = {
    'Score': scores,
    'Check-In': checkin,
    'Check-Out': checkout,
    'Nights': nights,
    'Comment_Date': com_date,
    'Count_Date': count_date,
    'Customer_Type': customer_type,
    'Country': d_country,
    'Travel_Purpose': trav_purpose,
    'Room_Name': room_names,
    'Positiv':pos,
    'Negativ':neg
}

df_1 = pd.DataFrame(df_1_base)

In [ ]:
# Saving the DataFrame as Hotel_ID
df_1.to_excel('../'+str(id)+'/Hotel '+str(id)+'.xlsx')

# OpenAI API Bewertung

In [ ]:
#load all important libraries for the GPT-3o sentiment analysis
import http.client
import json
from datetime import datetime
import pandas as pd 
import matplotlib.pyplot as plt 
from openai import OpenAI
import re
import spacy

In [ ]:
#Load the Hotel_ID.xlsx and safe as data_frame_1
data_frame_1 = pd.read_excel('../'+str(id)+"/Hotel "+str(id)+".xlsx")

In [ ]:
#Subset of Data_Frame_1
df_pos=data_frame_1[["Positiv","Customer_Type","Travel_Purpose","Room_Name","Comment_Date"]].copy()
df_neg=data_frame_1[["Negativ","Customer_Type","Travel_Purpose","Room_Name","Comment_Date"]].copy()
data_frame_1_new=data_frame_1.copy()

In [ ]:
#OPENAI_KEY
OPENAI_API_KEY = Your_KEY #here comes openai api key
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
#Define Tag Classes for the sentiment analysis
tags = "Amenities (Entrance Area, Hotel Building etc.);Bar (Prices, Service, Beverages, Snacks etc.);Bathroom Cleanliness (Toilet, Shower,Towel,Bath etc.);Room Cleanliness (Bed, Furniture, Closet,Desk etc.);Comfort (Air Conditioning, Bed Size, Noise Level,Room Size, WiFi etc.);Food (Breakfast, Menu, Dinner etc.);Location (Distance to City Centre, Parking, Restaurants, Shopping, Sightseeing etc.);Wellness (Massage, Pool, Fitness, Sauna, etc.);Price;Service (Bar, Booking Process, Houskeeping, Reception etc.);Sonstiges (Alles was nicht zu den anderen Passt)"
#define the command for GPT-3o
command = "Evaluate the given sentence against the given categories and return a dictionary with the categories as keys and 1 for a match, 0 for no match. No line breaks in the answer. All categories need to be answered. Keys in double quotation mark."

## Positive Comment Analysis

In [ ]:
#Split the comments in sentences and save them
#in positive dataframe
txt_pos=[]
df_zw_pos=[]
df_new_pos=[]
#
tag_pos_sonst=[]
tag_pos_bad=[]
tag_pos_room=[]
tag_pos_amenities=[]
tag_pos_bar=[]
tag_pos_comfort=[]
tag_pos_food=[]
tag_pos_location=[]
tag_pos_wellness=[]
tag_pos_price=[]
tag_pos_service=[]
#
df1_tag_pos_sonst=[]
df1_tag_pos_bad=[]
df1_tag_pos_room=[]
df1_tag_pos_amenities=[]
df1_tag_pos_bar=[]
df1_tag_pos_comfort=[]
df1_tag_pos_food=[]
df1_tag_pos_location=[]
df1_tag_pos_wellness=[]
df1_tag_pos_price=[]
df1_tag_pos_service=[]
#
for i in range(len(df_pos)):
    txt_pos.append(re.split(r'(?<=[.!?,-]) +', str(df_pos["Positiv"][i])))
    for j in range(len(txt_pos[i])):
        #print(txt_pos[i][j])<--only for testing
        if (txt_pos[i][j] != 'nan'):
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages = [
                {"role": "system", "content": "You are a text classification assistant. Multiple Fits are possible. Answer please in a format unsing the categories as key and 1 for positive and 0 for negative. Dont use linebreaks in the answer."},
                {"role": "user", "content": command+ "Categories:"+ tags+ "Sentence:" +str(txt_pos[i][j])}
                ]
            )
            answer=(completion.choices[0].message.content)
            #print(answer)<--only for testing
            answer_json = json.loads(answer)
            tag_pos_sonst.append(answer_json["Sonstiges"])
            tag_pos_amenities.append(answer_json["Amenities"])
            tag_pos_bar.append(answer_json["Bar"])
            tag_pos_room.append(answer_json["Room Cleanliness"])
            tag_pos_bad.append(answer_json["Bathroom Cleanliness"])
            tag_pos_comfort.append(answer_json["Comfort"])
            tag_pos_food.append(answer_json["Food"])
            tag_pos_location.append(answer_json["Location"])
            tag_pos_wellness.append(answer_json["Wellness"])
            tag_pos_price.append(answer_json["Price"])
            tag_pos_service.append(answer_json["Service"])
        else:
            tag_pos_sonst.append(float('nan'))
            tag_pos_amenities.append(float('nan'))
            tag_pos_bar.append(float('nan'))
            tag_pos_room.append(float('nan'))
            tag_pos_bad.append(float('nan'))
            tag_pos_comfort.append(float('nan'))
            tag_pos_food.append(float('nan'))
            tag_pos_location.append(float('nan'))
            tag_pos_wellness.append(float('nan'))
            tag_pos_price.append(float('nan'))
            tag_pos_service.append(float('nan'))
    df_zw_pos ={
        'Positiv': txt_pos[i],
        'Customer_Type': [df_pos.at[i,'Customer_Type']]*len(txt_pos[i]),
        'Travel_Purpose': [df_pos.at[i,'Travel_Purpose']]*len(txt_pos[i]),
        'Date': [df_pos.at[i,'Comment_Date']]*len(txt_pos[i]),
        'Room_Name': [df_pos.at[i,'Room_Name']]*len(txt_pos[i]),
        'Pos_Sonst': tag_pos_sonst,
        'Pos_Bad':tag_pos_bad,
        'Pos_Room':tag_pos_room,
        'Pos_Amenities':tag_pos_amenities,
        'Pos_Bar':tag_pos_bar,
        'Pos_Comfort':tag_pos_comfort,
        'Pos_Food':tag_pos_food,
        'Pos_Location':tag_pos_location,
        'Pos_Wellness':tag_pos_wellness,
        'Pos_Price':tag_pos_price,
        'Pos_Service':tag_pos_service
    }
    df1_tag_pos_sonst.append(max(tag_pos_sonst))
    df1_tag_pos_bad.append(max(tag_pos_bad))
    df1_tag_pos_room.append(max(tag_pos_room))
    df1_tag_pos_amenities.append(max(tag_pos_amenities))
    df1_tag_pos_bar.append(max(tag_pos_bar))
    df1_tag_pos_comfort.append(max(tag_pos_comfort))
    df1_tag_pos_food.append(max(tag_pos_food))
    df1_tag_pos_location.append(max(tag_pos_location))
    df1_tag_pos_wellness.append(max(tag_pos_wellness))
    df1_tag_pos_price.append(max(tag_pos_price))
    df1_tag_pos_service.append(max(tag_pos_service))
    #create the list, which will be later the dataframe
    df_new_pos.append(df_zw_pos)
    #reset the values
    tag_pos_sonst=[]
    tag_pos_bad=[]
    tag_pos_room=[]
    tag_pos_amenities=[]
    tag_pos_bar=[]
    tag_pos_comfort=[]
    tag_pos_food=[]
    tag_pos_location=[]
    tag_pos_wellness=[]
    tag_pos_price=[]
    tag_pos_service=[]
#create from the list a dataframe
df_new_pos_1 = pd.DataFrame(df_new_pos)

In [ ]:
#Flat the elements of the DataFrame
df_new_pos_1 = df_new_pos_1.apply(lambda col: col.apply(lambda x: x[0] if isinstance(x, list) else x))

In [ ]:
# save the new dataframe
name='../'+str(id)+"/Pos "+str(id)
df_new_pos_1.to_excel(name+".xlsx")

## Negative Comment Analysis

In [ ]:
#split comments in sentences and save in negative dataframe
txt_neg=[]
df_zw_neg=[]
df_new_neg=[]
#
tag_neg_sonst=[]
tag_neg_bad=[]
tag_neg_room=[]
tag_neg_amenities=[]
tag_neg_bar=[]
tag_neg_comfort=[]
tag_neg_food=[]
tag_neg_location=[]
tag_neg_wellness=[]
tag_neg_price=[]
tag_neg_service=[]
#
df1_tag_neg_sonst=[]
df1_tag_neg_bad=[]
df1_tag_neg_room=[]
df1_tag_neg_amenities=[]
df1_tag_neg_bar=[]
df1_tag_neg_comfort=[]
df1_tag_neg_food=[]
df1_tag_neg_location=[]
df1_tag_neg_wellness=[]
df1_tag_neg_price=[]
df1_tag_neg_service=[]
#
for i in range(len(df_neg)):
    txt_neg.append(re.split(r'(?<=[.!?,-]) +', str(df_neg["Negativ"][i])))
    for j in range(len(txt_neg[i])):
        #print(txt_pos[i][j])
        if (txt_neg[i][j] != 'nan'):
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages = [
                {"role": "system", "content": "You are a text classification assistant. Multiple Fits are possible. Answer please in a format unsing the categories as key and 1 for positive and 0 for negative. Dont use linebreaks in the answer."},
                {"role": "user", "content": command+ "Categories:"+ tags+ "Sentence:" +str(txt_neg[i][j])}
                ]
            )
            answer=(completion.choices[0].message.content)
            #print(answer)
            answer_json = json.loads(answer)
            tag_neg_sonst.append(answer_json["Sonstiges"])
            tag_neg_amenities.append(answer_json["Amenities"])
            tag_neg_bar.append(answer_json["Bar"])
            tag_neg_room.append(answer_json["Room Cleanliness"])
            tag_neg_bad.append(answer_json["Bathroom Cleanliness"])
            tag_neg_comfort.append(answer_json["Comfort"])
            tag_neg_food.append(answer_json["Food"])
            tag_neg_location.append(answer_json["Location"])
            tag_neg_wellness.append(answer_json["Wellness"])
            tag_neg_price.append(answer_json["Price"])
            tag_neg_service.append(answer_json["Service"])
        else:
            tag_neg_sonst.append(float('nan'))
            tag_neg_amenities.append(float('nan'))
            tag_neg_bar.append(float('nan'))
            tag_neg_room.append(float('nan'))
            tag_neg_bad.append(float('nan'))
            tag_neg_comfort.append(float('nan'))
            tag_neg_food.append(float('nan'))
            tag_neg_location.append(float('nan'))
            tag_neg_wellness.append(float('nan'))
            tag_neg_price.append(float('nan'))
            tag_neg_service.append(float('nan'))
    df_zw_neg ={
        'Negativ': txt_neg[i],
        'Customer_Type': [df_neg.at[i,'Customer_Type']]*len(txt_neg[i]),
        'Travel_Purpose': [df_neg.at[i,'Travel_Purpose']]*len(txt_neg[i]),
        'Date': [df_neg.at[i,'Comment_Date']]*len(txt_neg[i]),
        'Room_Name': [df_neg.at[i,'Room_Name']]*len(txt_neg[i]),
        'Neg_Sonst': tag_neg_sonst,
        'Neg_Bad':tag_neg_bad,
        'Neg_Room':tag_neg_room,
        'Neg_Amenities':tag_neg_amenities,
        'Neg_Bar':tag_neg_bar,
        'Neg_Comfort':tag_neg_comfort,
        'Neg_Food':tag_neg_food,
        'Neg_Location':tag_neg_location,
        'Neg_Wellness':tag_neg_wellness,
        'Neg_Price':tag_neg_price,
        'Neg_Service':tag_neg_service
    }
    df1_tag_neg_sonst.append(max(tag_neg_sonst))
    df1_tag_neg_bad.append(max(tag_neg_bad))
    df1_tag_neg_room.append(max(tag_neg_room))
    df1_tag_neg_amenities.append(max(tag_neg_amenities))
    df1_tag_neg_bar.append(max(tag_neg_bar))
    df1_tag_neg_comfort.append(max(tag_neg_comfort))
    df1_tag_neg_food.append(max(tag_neg_food))
    df1_tag_neg_location.append(max(tag_neg_location))
    df1_tag_neg_wellness.append(max(tag_neg_wellness))
    df1_tag_neg_price.append(max(tag_neg_price))
    df1_tag_neg_service.append(max(tag_neg_service))
    #create the list, which will be later the dataframe
    df_new_neg.append(df_zw_neg)
    #reset the values
    tag_neg_sonst=[]
    tag_neg_bad=[]
    tag_neg_room=[]
    tag_neg_amenities=[]
    tag_neg_bar=[]
    tag_neg_comfort=[]
    tag_neg_food=[]
    tag_neg_location=[]
    tag_neg_wellness=[]
    tag_neg_price=[]
    tag_neg_service=[]
#create a dataframe from the list
df_new_neg_1 = pd.DataFrame(df_new_neg)

In [ ]:
#Flat the elements of the dataframe
df_new_neg_1 = df_new_neg_1.apply(lambda col: col.apply(lambda x: x[0] if isinstance(x, list) else x))

In [ ]:
# save the dataframe
name='../'+(id)+"/Neg "+str(id)
df_new_neg_1.to_excel(name+".xlsx")

## Generate a General.xlsx including the tags

In [ ]:
# Add the tags to a general dataframe
data_frame_1_new["Neg_Sonst"]=df1_tag_neg_sonst
data_frame_1_new["Neg_Bad"]=df1_tag_neg_bad
data_frame_1_new["Neg_Room"]=df1_tag_neg_room
data_frame_1_new["Neg_Amenities"]=df1_tag_neg_amenities
data_frame_1_new["Neg_Bar"]=df1_tag_neg_bar
data_frame_1_new["Neg_Comfort"]=df1_tag_neg_comfort
data_frame_1_new["Neg_Food"]=df1_tag_neg_food
data_frame_1_new["Neg_Location"]=df1_tag_neg_location
data_frame_1_new["Neg_Wellness"]=df1_tag_neg_wellness
data_frame_1_new["Neg_Price"]=df1_tag_neg_price
data_frame_1_new["Neg_Service"]=df1_tag_neg_service
data_frame_1_new["Pos_Sonst"]=df1_tag_pos_sonst
data_frame_1_new["Pos_Bad"]=df1_tag_pos_bad
data_frame_1_new["Pos_Room"]=df1_tag_pos_room
data_frame_1_new["Pos_Amenities"]=df1_tag_pos_amenities
data_frame_1_new["Pos_Bar"]=df1_tag_pos_bar
data_frame_1_new["Pos_Comfort"]=df1_tag_pos_comfort
data_frame_1_new["Pos_Food"]=df1_tag_pos_food
data_frame_1_new["Pos_Location"]=df1_tag_pos_location
data_frame_1_new["Pos_Wellness"]=df1_tag_pos_wellness
data_frame_1_new["Pos_Price"]=df1_tag_pos_price
data_frame_1_new["Pos_Service"]=df1_tag_pos_service

In [ ]:
# safe the dataframe
name='../'+str(id)+"/General "+str(id)
data_frame_1_new.to_excel(name+".xlsx")